In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Tikhonov regularisation using CGLS and block framework
Few lines intro

**Learning objectives:**
1. Construct and manipulate BlockOperators and BlockDataContainer, including direct and adjoint operations and algebra.
2. Use Block Framework to solve Tikhonov regularisation with CGLS algorithm.
3. Apply Tikhonov regularisation to tomographic reconstruction and explain the effect of regularization parameter and operator in regulariser.

In [ ]:
#imports
from ccpi.framework import ImageGeometry, ImageData 
from ccpi.framework import AcquisitionGeometry, AcquisitionData
from ccpi.framework import BlockDataContainer

from ccpi.optimisation.algorithms import CGLS
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity

from ccpi.astra.operators import AstraProjectorSimple 

import astra.functions

import tomophantom
from tomophantom import TomoP2D

import numpy as np                          
import matplotlib.pyplot as plt

import os

In [ ]:
def plot2D(datacontainer1, title1, datacontainer2, title2):
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,5))
    plt.subplots_adjust(wspace = 0.5)
    
    ax1.set_title(title1)
    subplot1 = ax1.imshow(datacontainer1.as_array())
    plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)

    ax2.set_title(title2)
    subplot2 = ax2.imshow(datacontainer2.as_array())
    plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)

    a, b = ax1.get_xlim()
    c, d = ax1.get_ylim()
    if(abs(b - a) != abs(d - c)):
        ax1.set_aspect('auto')
        ax2.set_aspect('auto')
        
    plt.show()

### Setting up the dataset - 2D

In [ ]:
#set up acquisition geometry
number_pixels_x = 1024
number_projections = 180
angles = np.linspace(0, np.pi, number_projections, dtype=np.float32)
ag = AcquisitionGeometry(geom_type='parallel', dimension='2D', angles=angles, pixel_num_h=number_pixels_x)

#set up image geometry
num_voxels_xy = 1024
ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)

In [ ]:
# Load Shepp-Logan phantom 
model = 1
path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")

#tomophantom takes angular input in degrees
phantom_2D = TomoP2D.Model(model, num_voxels_xy, path_library2D)
phantom_sino = TomoP2D.ModelSino(model, num_voxels_xy, number_pixels_x, angles*180./np.pi, path_library2D)

sinogram = AcquisitionData(phantom_sino)

In [ ]:
#add Poisson noise to the sinogram
data_noisy = astra.functions.add_noise_to_sino(sinogram.as_array(),400)

sinogram_noisy = ag.allocate()
sinogram_noisy.fill(data_noisy)

In [ ]:
plot2D(sinogram, "sinogram", sinogram_noisy, "sinogram noisy")

<a id="section_CGLS_simple"></a>
### Reconstruct using unregularised CGLS

Solve:
$$\underset{x}{\mathrm{argmin}}\begin{Vmatrix}A x - b\end{Vmatrix}^2_2$$

where,

- $A$ is the projection operator

- $b$ is the acquired data

- $x$ is the solution

Reconstruct dataset using CGLS


In [ ]:
#define the operator A
device = "gpu"
operator = AstraProjectorSimple(ig, ag, device)

In [ ]:
#define the data b
data = sinogram_noisy

In [ ]:
#setup CGLS
x_init = ig.allocate()
cgls = CGLS(x_init=x_init, operator=operator, data=data)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(1000, verbose = True)

In [ ]:
#plot the results
residuals = ig.allocate()
residuals.fill(cgls.get_output().as_array()- phantom_2D)
plot2D(cgls.get_output(), "CGLS reconstruction", residuals, "Residuals")

**Exercise: Repeat this with the noisy dataset** [go to section start](#section_CGLS_simple)

### Reconstruct using regularised CGLS

#### Regularisation

Noisy datasets lead to an ill-posed problem. If we try to solve these using LS we end up with a noisy reconstruction. Regularisation adds information in order for us to solve the problem.

#### CGLS and regularisation

Adding a differentiable regulariser...
Identity
gradient



Solve:
$$\underset{x}{\mathrm{argmin}}\begin{Vmatrix}A x - b \end{Vmatrix}^2_2 + \alpha\|Lx\|^2_2$$


where,

- $A$ is the projection operator

- $b$ is the acquired data

- $x$ is the solution

- $\alpha$ is the regularisation parameter

- $L$ is a regularisation operator

<br>This can be re-written in the form:

$$\underset{x}{\mathrm{argmin}}\begin{Vmatrix}\binom{A}{\alpha L} x - \binom{b}{0}\end{Vmatrix}^2_2$$

Which allows us to solve it using CGLS in the form:

$$\underset{x}{\mathrm{argmin}}\begin{Vmatrix}\tilde{A} x - \tilde{b}\end{Vmatrix}^2_2$$

where:

- $\tilde{A} = \binom{A}{\alpha L}$

- $\tilde{b} = \binom{b}{0}$



#### Introducing the block framework

We can construct $\tilde{A}$ and $\tilde{b}$ using the BlockFramework in the CIL.

$\tilde{A}$ is a BlockOperator

$\tilde{b}$ is a BlockDataContainer

<a id="section_CGLS_alpha"></a>
#### Reconstruct using CGLS and the identity operator

In [ ]:
#define the operator A
device = "gpu"
A = AstraProjectorSimple(ig, ag, device)
L = Identity(ig)
alpha = 2

operator_block = BlockOperator( A, alpha * L, shape=(2,1))

In [ ]:
#define the data b
data_block = BlockDataContainer(sinogram_noisy, L.range_geometry().allocate())

Run CGLS as before, but passing the BlockOperator and BlockDataContainer

In [ ]:
#setup CGLS with the Block Operator and Block DataContainer
x_init = ig.allocate()      
cgls = CGLS(x_init=x_init, operator=operator_block, data=data_block)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(1000, verbose = True)

In [ ]:
residuals = ig.allocate()
residuals.fill(cgls.get_output().as_array()- phantom_2D)
plot2D(cgls.get_output(), "CGLS reconstruction", residuals, "Residuals")

**Exercise: Repeat this with more regularisation** [go to section start](#section_CGLS_alpha)

### A more detailed look at the BlockFramework

In [ ]:
#more general examples

In [ ]:
#block datacontainers

In [ ]:
#blockoperators

In [ ]:
#the gradient operator

#blockoperator
#direct takes an ig and returns a datacontainer of dim*ig
#adjoint runs on the datacontainer and returns an ig

#specify the directions to run over

### Reconstruct using regularised CGLS with Tikhonov regularisation

#### Tikhonov regularisation

$$ \underset{x}{\mathrm{argmin}}\begin{Vmatrix}A x - b \end{Vmatrix}^2_2 + \alpha\begin{Vmatrix}\nabla x\end{Vmatrix}^2_2$$



In [ ]:
#define the operator A
device = "gpu"
A = AstraProjectorSimple(ig, ag, device)
L = Gradient(ig)
alpha = 50

operator_block = BlockOperator( A, alpha * L, shape=(2,1))

In [ ]:
#define the data b
data_block = BlockDataContainer(sinogram_noisy, L.range_geometry().allocate())

In [ ]:
#setup CGLS with the block operator and block data
x_init = ig.allocate()      
cgls = CGLS(x_init=x_init, operator=operator_block, data=data_block)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(1000, verbose = True)

In [ ]:
residuals = ig.allocate()
residuals.fill(cgls.get_output().as_array()- phantom_2D)
plot2D(cgls.get_output(), "CGLS reconstruction", residuals, "Residuals")

### A 3D example

In [ ]:
#diamond dataset

##set up gradient in x,y and z

### Summary